In [42]:
import numpy as np
import pandas as pd
import string
from gensim.models import Word2Vec

In [9]:
data_main = pd.read_csv('../../raw_data/aggression_parsed_dataset.csv',usecols=['Text','oh_label'])
data_main.head()

,Text,oh_label
0,`- This is not ``creative``. Those are the di...,0
1,` :: the term ``standard model`` is itself le...,0
2,"True or false, the situation as of March 200...",0
3,"Next, maybe you could work on being less cond...",0
4,This page will need disambiguation.,0


In [28]:
data_main.shape

(115864, 2)

In [14]:
data_twitter = pd.read_csv('../../raw_data/twitter_parsed_dataset.csv',usecols=['Text','oh_label'])
data_twitter.head()

,Text,oh_label
0,@halalflaws @biebervalue @greenlinerzjm I read...,0.0
1,@ShreyaBafna3 Now you idiots claim that people...,0.0
2,"RT @Mooseoftorment Call me sexist, but when I ...",1.0
3,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",1.0
4,#mkr No No No No No No,0.0


In [29]:
data_twitter.shape

(16851, 2)

In [19]:
data_youtube = pd.read_csv('../../raw_data/youtube_parsed_dataset.csv',usecols=['Text','oh_label'])
data_youtube.head()

,Text,oh_label
0,Does N.e.bodyelse Hear her Crazy ass Screamin ...,0
1,There are so many things that are incorrect wi...,0
2,3:26 hahah my boyfriend showed this song to me...,1
3,dick beyonce fuck y a ass hole you are truely ...,1
4,DongHaeTaemin and Kai ;A; luhansehun and bacon...,0


In [20]:
data_kaggle = pd.read_csv('../../raw_data/kaggle_parsed_dataset.csv',usecols=['Text','oh_label'])
data_kaggle.head()

,oh_label,Text
0,1,"""You fuck your dad."""
1,0,"""i really don't understand your point.\xa0 It ..."
2,0,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,"""listen if you dont wanna get married to a man..."
4,0,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [21]:
data_toxi = pd.read_csv('../../raw_data/toxicity_parsed_dataset.csv',usecols=['Text','oh_label'])
data_toxi.head()

,Text,oh_label
0,This: :One can make an analogy in mathematical...,0
1,` :Clarification for you (and Zundark's righ...,0
2,Elected or Electoral? JHK,0
3,`This is such a fun entry. Devotchka I once...,0
4,Please relate the ozone hole to increases in c...,0


In [31]:
data_toxi.shape

(159686, 2)

In [24]:
data_attack = pd.read_csv('../../raw_data/attack_parsed_dataset.csv',usecols=['Text','oh_label'])
data_attack.shape

(115864, 2)

In [30]:
data = pd.concat([data_main,data_twitter,data_youtube,data_kaggle,data_toxi])
data.shape

(304664, 2)

In [32]:
def clean_data(text):
    
    text = str(text)
    
    # remove punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    # lower text
    text = text.lower()
    
    # remove numbers
    text = ''.join([w for w in text if not w.isdigit()]) 
    
    return text


In [36]:
data['clean_text'] = data['Text'].apply(clean_data)

In [44]:
data.rename(columns={'Text':'text','oh_label':'target'},inplace=True)

In [45]:
data.head()

,text,target,clean_text
0,`- This is not ``creative``. Those are the di...,0.0,this is not creative those are the dictionar...
1,` :: the term ``standard model`` is itself le...,0.0,the term standard model is itself less npov...
2,"True or false, the situation as of March 200...",0.0,true or false the situation as of march was...
3,"Next, maybe you could work on being less cond...",0.0,next maybe you could work on being less conde...
4,This page will need disambiguation.,0.0,this page will need disambiguation


In [41]:
data.describe()

,oh_label
count,304661.000000
mean,0.127072
std,0.333055
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [50]:
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import train_test_split

def load_data(percentage_of_sentences=None):
    
    X_train, X_test, y_train, y_test = train_test_split(data['clean_text'],data['target'],test_size=0.3,random_state=0)

    #train_sentences, y_train = tfds.as_numpy(train_data)
    #test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(X_train))
        X_train, y_train = X_train[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(X_test))
        X_test, y_test = X_test[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_) for _ in X_train]
    X_test = [text_to_word_sequence(_) for _ in X_test]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


In [52]:
word2vec = Word2Vec(sentences=X_train, vector_size=10)


In [54]:
comm_1 = word2vec.wv.most_similar('asshole')
comm_1

[('kiss', 0.9767996668815613),
 ('pussy', 0.9634009599685669),
 ('cunt', 0.9619564414024353),
 ('suck', 0.9595346450805664),
 ('fuckin', 0.9548425078392029),
 ('fuck', 0.9518783092498779),
 ('dumb', 0.9504834413528442),
 ('fuk', 0.948442816734314),
 ('fucking', 0.9451690912246704),
 ('retard', 0.9413929581642151)]

In [57]:
vocab_size = len(word2vec.wv.key_to_index)
vocab_size

12693

In [59]:
data = data.drop_duplicates()

In [60]:
data.describe()

,target
count,230949.000000
mean,0.119048
std,0.323846
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [61]:
data.to_csv('../../cyberbullying/data/data.csv')